# 第14章：もうちょっとだけモナド

Maybeモナド・リストもなど・IOモナドを今までやってきた  
さらにいくつかのモナドがこの章で出てくる  

この章で出てくるのはmtlパッケージ  
インストールされているかは `ghc-pkg list` とうってmtlとバージョン番号が出るはず  

### Writerの話

In [1]:
isBigGang :: Int -> Bool
isBigGang x = x > 9

In [2]:
isBigGang :: Int -> (Bool, String)
isBigGang x = (x > 9, "Compared gang size to 9.")

In [3]:
isBigGang 3

(False,"Compared gang size to 9.")

例えば既に文脈がついた値 `(3, "Smallish gang.")` をisBigGangに食わせたかったらどうするか  
ログのついた値、`(a, String)` 型の値と `a -> (b, String)` 型の関数2つを取り、その値を関数の方に食わせる関数を作る  

In [4]:
applyLog :: (a, String) -> (a -> (b, String)) -> (b, String)
applyLog (x, log) f = let (y, newLog) = f x in (y, log ++ newLog)

In [6]:
(3, "Smallish gang.") `applyLog` isBigGang

(False,"Smallish gang.Compared gang size to 9.")

In [7]:
("Tobin", "Got outlaw name.") `applyLog` (\x -> (length x, "applied length"))

(5,"Got outlaw name.applied length")

### モノイドが助けに来たよ

Logは別にStringである必要はない  


In [1]:
applyLog :: (Monoid m) => (a, m) -> (a -> (b, m)) -> (b, m)
applyLog (x, log) f = let (y, newLog) = f x in (y, log `mappend` newLog)


In [4]:
import Data.Monoid

type Food = String
type Price = Sum Int

addDrink :: Food -> (Food, Price)
addDrink "beans" = ("milk", Sum 25)
addDrink "jerky" = ("whiskey", Sum 99)
addDrink _ = ("beer", Sum 30)


In [3]:
Sum 3 `mappend` Sum 9

Not in scope: data constructor `Sum'
Perhaps you meant variable `sum' (imported from Prelude)


Not in scope: data constructor `Sum'
Perhaps you meant variable `sum' (imported from Prelude)

In [2]:
("beans", Sum 10) `applyLog` addDrink

Not in scope: data constructor `Sum'
Perhaps you meant variable `sum' (imported from Prelude)


Not in scope: `addDrink'

### Writer型

```
newtype Writer w a = Writer {runWriter :: (a, w)}
```

In [7]:
import Control.Monad.Writer

In [8]:
runWriter (return 3 :: Writer String Int)

(3,"")

In [9]:
runWriter (return 3 :: Writer String Int)

(3,"")

In [10]:
    runWriter (return 3 :: Writer (Product Int) Int)

(3,Product {getProduct = 1})

In [11]:
logNumber :: Int -> Writer [String] Int 
logNumber x = writer (x, ["Got number: " ++ show x]) 

multWithLog :: Writer [String] Int
multWithLog = do 
    a <- logNumber 3 
    b <- logNumber 5 
    return (a* b)

In [12]:
logNumber 1

WriterT (Identity (1,["Got number: 1"]))

In [14]:
runWriter multWithLog

(15,["Got number: 3","Got number: 5"])

In [15]:
multWithLog :: Writer [String] Int
multWithLog = do
    a <- logNumber 3 
    b <- logNumber 5 
    tell ["Gonna multiply these two"] 
    return (a* b)

In [16]:
runWriter multWithLog

(15,["Got number: 3","Got number: 5","Gonna multiply these two"])

In [17]:
gcd' :: Int -> Int -> Writer [String] Int

gcd' a b
    | b == 0 = do
        tell ["Finished with " ++ show a] 
        return a
    | otherwise = do
        tell [show a ++ " mod " ++ show b ++ " = " ++ show (a `mod ` b)] 
        gcd' b (a ` mod ` b)

In [18]:
gcd' 8 3

WriterT (Identity (1,["8 mod 3 = 2","3 mod 2 = 1","2 mod 1 = 0","Finished with 1"]))

In [19]:
fst $ runWriter (gcd' 8 3)

1

In [20]:
mapM_ putStrLn $ snd $ runWriter (gcd' 8 3)

8 mod 3 = 2
3 mod 2 = 1
2 mod 1 = 0
Finished with 1

### 非効率なリスト構築

In [22]:
gcdReverse :: Int -> Int -> Writer [String] Int 
gcdReverse a b
    | b == 0 = do 
        tell ["Finished with " ++ show a] 
        return a 
    | otherwise = do 
        result <- gcdReverse b (a ` mod ` b) 
        tell [show a ++ " mod " ++ show b   ++ " = " ++  show (a ` mod ` b)] 
        return result
        

In [23]:
mapM_ putStrLn $ snd $ runWriter (gcdReverse 8 3)

Finished with 1
2 mod 1 = 0
3 mod 2 = 1
8 mod 3 = 2

In [24]:
newtype DiffList a = DiffList {getDiffList :: [a] -> [a]}

In [25]:
toDiffList :: [a] -> DiffList a
toDiffList xs = DiffList (xs++)

fromDiffList :: DiffList a -> [a]
fromDiffList (DiffList f) = f []

In [26]:
instance Monoid (DiffList a) where
    mempty = DiffList (\xs -> [] ++ xs)
    (DiffList f) `mappend` (DiffList g) = DiffList (\xs -> f (g xs))

Line 3: Avoid lambda
Found:
\ xs -> f (g xs)
Why not:
f . g

In [27]:
fromDiffList (toDiffList [1, 2, 3, 4] ` mappend ` toDiffList [1, 2, 3])

[1,2,3,4,1,2,3]

In [30]:
gcd' :: Int -> Int -> Writer (DiffList String) Int
gcd' a b
    | b == 0 = do 
        tell (toDiffList ["Finished with " ++ show a]) 
        return a 
    | otherwise = do 
        result <- gcd' b (a ` mod ` b)
        tell (toDiffList [show a ++ " mod " ++ show b   ++ " = " ++  show (a ` mod ` b)] )
        return result

In [32]:
mapM_ putStrLn . fromDiffList . snd . runWriter $ gcd' 110 34

Finished with 2
8 mod 2 = 0
34 mod 8 = 2
110 mod 34 = 8